<h3>Import Libraries and Data</h3>

In [28]:
import pandas as pd
from sqlalchemy import create_engine

In [29]:
# Store csv data into DataFrame
csv_file = "./data_sources/combined_data_clean.csv"
combined_data_df = pd.read_csv(csv_file)
combined_data_df.head()

,Name,county_state,date1,Serial,date2,Pre_rally_beg,pre_rally_end,post_rally_beg,post_rally_end,average_growth_new_cases,pre_rally_rate,post_rally_rate,post_rally_growth_new_cases,post_rally_change_in_growth,pre_rally_rate.1,post_rally_rate.1,post_rally_growth_deaths
0,Joe Biden,Milwaukee_Wisconsin,20200903,Milwaukee_Wisconsin_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,8.80%,330503,399423,20.85%,136.93%,6777,7619,12.42%
1,Donald Trump,Westmoreland_Pennsylvania,20200903,Westmoreland_Pennsylvania_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,28.60%,24721,28982,17.24%,-39.72%,686,736,7.29%
2,Joe Biden,Dauphin_Pennsylvania,20200907,Dauphin_Pennsylvania_20200907,9/7/2020,8/24/2020,9/7/2020,9/7/2020,9/21/2020,-2.50%,46296,53798,16.20%,748.00%,2284,2543,11.34%
3,Donald Trump,Forsyth_North Carolina,20200908,Forsyth_North Carolina_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-3.40%,86603,99921,15.38%,552.35%,1068,1342,25.66%
4,Donald Trump,Palm Beach_Florida,20200908,Palm Beach_Florida_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-11.90%,589620,665859,12.93%,208.66%,15758,18412,16.84%


<h3>Clean and Transform Data</h3>

In [30]:
# Rename combined_data_df columns to match schema
combined_data_df.rename(columns={'Name': 'name', 'date2': 'date', 'Pre_rally_beg': 'pre_rally_date', 
                                'post_rally_end': 'post_rally_date', 'average_growth_new_cases': 'pre_rally_case_growth_rate',
                                'pre_rally_rate': 'pre_rally_num_cases', 'post_rally_rate': 'post_rally_num_cases',
                                'post_rally_growth_new_cases': 'post_rally_case_growth_rate',
                                'post_rally_change_in_growth': 'case_growth_rate_change',
                                'pre_rally_rate.1': 'pre_rally_num_deaths',
                                'post_rally_rate.1': 'post_rally_num_deaths',
                                'post_rally_growth_deaths': 'death_growth_rate_change'}, inplace=True)
# Convert percent to float
combined_data_df['pre_rally_case_growth_rate'] = combined_data_df['pre_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['post_rally_case_growth_rate'] = combined_data_df['post_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['case_growth_rate_change'] = combined_data_df['case_growth_rate_change'].str.rstrip('%').astype('float') / 100.0
combined_data_df['death_growth_rate_change'] = combined_data_df['death_growth_rate_change'].str.rstrip('%').astype('float') / 100.0

# print(combined_data_df)

<h3>Normalize Candidates</h3>

In [31]:
rally_df = pd.DataFrame() # Initialize to lock printed table on rerun
rally_df = combined_data_df[['county_state', 'date', 'case_growth_rate_change', 'death_growth_rate_change']].copy()

rally_candidate_df = combined_data_df[['name']].copy()
print(rally_candidate_df)

candidate_list = []
rally_candidate = []


# Candidates are NOT hard-coded, we allow for any variable number of candidates.

# Loop through candidate_df, if current candidate is not in candidate list, add to list.
# Add and populate new candidate_ID column in rally_df.
for row in rally_candidate_df.itertuples():
    if not row[1] in candidate_list: # If candidate is not already in candidate_list,
        candidate_list.append(row[1]) # Then append candidate to candidate_list.
    rally_candidate.append(candidate_list.index(row[1]) + 1) # Track candidate_id foreign key. +1 because pgAdmin candidate table index starts at 1.

rally_df = rally_df.assign(candidate_id = rally_candidate) # New column. Assign rally candidate, Biden or Trump, to rally.

# print(candidate_list)

candidate_df = pd.DataFrame(candidate_list,columns=['name'])

# print(rally_candidate)
print(candidate_df)

rally_df.head()

             name
0       Joe Biden
1    Donald Trump
2       Joe Biden
3    Donald Trump
4    Donald Trump
..            ...
102  Donald Trump
103  Donald Trump
104  Donald Trump
105     Joe Biden
106     Joe Biden

[107 rows x 1 columns]
           name
0     Joe Biden
1  Donald Trump


,county_state,date,case_growth_rate_change,death_growth_rate_change,candidate_id
0,Milwaukee_Wisconsin,9/3/2020,1.3693,0.1242,1
1,Westmoreland_Pennsylvania,9/3/2020,-0.3972,0.0729,2
2,Dauphin_Pennsylvania,9/7/2020,7.4800,0.1134,1
3,Forsyth_North Carolina,9/8/2020,5.5235,0.2566,2
4,Palm Beach_Florida,9/8/2020,2.0866,0.1684,2


<h3>Connect to Local Database</h3>

In [32]:
rds_connection_string = "postgres:password!@localhost:5432/covid_campaign" # Must have existing 'covid_campaign' database in pgAdmin
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['candidate', 'rally', 'pre_rally_covid_stats', 'post_rally_covid_stats']

<h3>Create Database Tables</h3>

Execute the following SQL code through the query editor in pgAdmin:
``` mysql
DROP TABLE IF EXISTS candidate CASCADE;
DROP TABLE IF EXISTS pre_rally_covid_stats CASCADE;
DROP TABLE IF EXISTS post_rally_covid_stats CASCADE;
DROP TABLE IF EXISTS rally CASCADE;

CREATE TABLE candidate (
	id SERIAL PRIMARY KEY,
	name TEXT
);

CREATE TABLE pre_rally_covid_stats (
	id SERIAL PRIMARY KEY,
	pre_rally_date DATE, -- Pre window start date
	pre_rally_num_cases INT, -- Number COVID-19 cases on day of rally
	pre_rally_case_growth_rate DECIMAL,
	pre_rally_num_deaths INT -- Number COVID-19 deaths on day of rally
);

CREATE TABLE post_rally_covid_stats (
	id SERIAL PRIMARY KEY,
	post_rally_date DATE, -- Post window end date
	post_rally_num_cases INT,
	post_rally_case_growth_rate DECIMAL,
	post_rally_num_deaths INT
);

CREATE TABLE rally (
	id SERIAL PRIMARY KEY,
	candidate_id INT,
	county_state TEXT,
	date DATE, -- Date of rally, end of pre window, start of post window
	pre_rally_covid_stats_id INT,
	post_rally_covid_stats_id INT,
	case_growth_rate_change DECIMAL,
	death_growth_rate_change DECIMAL,
    FOREIGN KEY (candidate_id)
        REFERENCES candidate(id)
        ON DELETE CASCADE,
    FOREIGN KEY (pre_rally_covid_stats_id)
        REFERENCES pre_rally_covid_stats(id)
        ON DELETE CASCADE,
    FOREIGN KEY (post_rally_covid_stats_id)
        REFERENCES post_rally_covid_stats(id)
        ON DELETE CASCADE
);```

<h3>Build rally_df, pre_rally_covid_stats_df, post_rally_covid_stats_df</h3>

In [33]:
# Add foreign key columns
rally_df = rally_df.drop([rally_df.index[105], rally_df.index[70]]) # Drop two greatest outliers, one for each candidate

rally_df['pre_rally_covid_stats_id'] = range(1, len(rally_df) + 1)
rally_df['post_rally_covid_stats_id'] = range(1, len(rally_df) + 1) # pgAdmin table index starts at 1

rally_df.head()
# print(rally_df)

,county_state,date,case_growth_rate_change,death_growth_rate_change,candidate_id,pre_rally_covid_stats_id,post_rally_covid_stats_id
0,Milwaukee_Wisconsin,9/3/2020,1.3693,0.1242,1,1,1
1,Westmoreland_Pennsylvania,9/3/2020,-0.3972,0.0729,2,2,2
2,Dauphin_Pennsylvania,9/7/2020,7.4800,0.1134,1,3,3
3,Forsyth_North Carolina,9/8/2020,5.5235,0.2566,2,4,4
4,Palm Beach_Florida,9/8/2020,2.0866,0.1684,2,5,5


In [34]:
pre_rally_covid_stats_df = combined_data_df[['pre_rally_date', 'pre_rally_num_cases', 
                                             'pre_rally_case_growth_rate', 'pre_rally_num_deaths']].copy()

pre_rally_covid_stats_df = pre_rally_covid_stats_df.drop([pre_rally_covid_stats_df.index[105], pre_rally_covid_stats_df.index[70]]) # Drop two greatest outliers, one for each candidate

pre_rally_covid_stats_df.head()
# print(pre_rally_covid_stats_df)

,pre_rally_date,pre_rally_num_cases,pre_rally_case_growth_rate,pre_rally_num_deaths
0,8/20/2020,330503,0.088,6777
1,8/20/2020,24721,0.286,686
2,8/24/2020,46296,-0.025,2284
3,8/25/2020,86603,-0.034,1068
4,8/25/2020,589620,-0.119,15758


In [35]:
post_rally_covid_stats_df = combined_data_df[['post_rally_date', 'post_rally_num_cases', 
                                             'post_rally_case_growth_rate', 'post_rally_num_deaths']].copy()

post_rally_covid_stats_df = post_rally_covid_stats_df.drop([post_rally_covid_stats_df.index[105], post_rally_covid_stats_df.index[70]]) # Drop two greatest outliers, one for each candidate

post_rally_covid_stats_df.head()
# print(post_rally_covid_stats_df)

,post_rally_date,post_rally_num_cases,post_rally_case_growth_rate,post_rally_num_deaths
0,9/17/2020,399423,0.2085,7619
1,9/17/2020,28982,0.1724,736
2,9/21/2020,53798,0.1620,2543
3,9/22/2020,99921,0.1538,1342
4,9/22/2020,665859,0.1293,18412


<h3>Load DataFrames Into Database</h3>

In [36]:
# Load cleaned pre_stats and post_stats DataFrames into database
pre_rally_covid_stats_df.to_sql(name='pre_rally_covid_stats', con=engine, if_exists='append', index=False)
post_rally_covid_stats_df.to_sql(name='post_rally_covid_stats', con=engine, if_exists='append', index=False)

In [37]:
# Load cleaned rally and candidate DataFrames into database
candidate_df.to_sql(name='candidate', con=engine, if_exists='append', index=False)
rally_df.to_sql(name='rally', con=engine, if_exists='append', index=False)

<h3>Query Database and Confirm Load</h3>

In [38]:
pd.read_sql_query('SELECT * FROM candidate', con=engine)

,id,name
0,1,Joe Biden
1,2,Donald Trump


In [39]:
pd.read_sql_query('SELECT * FROM rally', con=engine)

,id,candidate_id,county_state,date,pre_rally_covid_stats_id,post_rally_covid_stats_id,case_growth_rate_change,death_growth_rate_change
0,1,1,Milwaukee_Wisconsin,2020-09-03,1,1,1.3693,0.1242
1,2,2,Westmoreland_Pennsylvania,2020-09-03,2,2,-0.3972,0.0729
2,3,1,Dauphin_Pennsylvania,2020-09-07,3,3,7.4800,0.1134
3,4,2,Forsyth_North Carolina,2020-09-08,4,4,5.5235,0.2566
4,5,2,Palm Beach_Florida,2020-09-08,5,5,2.0866,0.1684
...,...,...,...,...,...,...,...,...
100,101,2,Cumberland_North Carolina,2020-11-02,101,101,12.0875,0.1770
101,102,2,Grand Traverse_Michigan,2020-11-02,102,102,1.2038,0.1613
102,103,2,Kent_Michigan,2020-11-02,103,103,2.6897,0.2821
103,104,2,Luzerne_Pennsylvania,2020-11-02,104,104,16.4050,0.1813


In [40]:
pd.read_sql_query('SELECT * FROM pre_rally_covid_stats', con=engine)

,id,pre_rally_date,pre_rally_num_cases,pre_rally_case_growth_rate,pre_rally_num_deaths
0,1,2020-08-20,330503,0.088,6777
1,2,2020-08-20,24721,0.286,686
2,3,2020-08-24,46296,-0.025,2284
3,4,2020-08-25,86603,-0.034,1068
4,5,2020-08-25,589620,-0.119,15758
...,...,...,...,...,...
100,101,2020-10-19,96865,0.016,1345
101,102,2020-10-19,10785,0.235,186
102,103,2020-10-19,198458,0.146,2570
103,104,2020-10-19,72346,-0.020,2753


In [41]:
pd.read_sql_query('SELECT * FROM post_rally_covid_stats', con=engine)

,id,post_rally_date,post_rally_num_cases,post_rally_case_growth_rate,post_rally_num_deaths
0,1,2020-09-17,399423,0.2085,7619
1,2,2020-09-17,28982,0.1724,736
2,3,2020-09-21,53798,0.1620,2543
3,4,2020-09-22,99921,0.1538,1342
4,5,2020-09-22,665859,0.1293,18412
...,...,...,...,...,...
100,101,2020-11-16,117152,0.2094,1583
101,102,2020-11-16,16371,0.5179,216
102,103,2020-11-16,305367,0.5387,3295
103,104,2020-11-16,94639,0.3081,3252


<h3>Analysis</h3>

In [42]:
df = pd.read_sql_query('SELECT * FROM rally WHERE candidate_id = 1', con=engine) # Biden

print('Counties saw, on average, a {:.2%} increase in the rate of COVID-19 cases from before Biden\'s campaign rallies to after rallies.'
      .format(df["case_growth_rate_change"].mean()))
print('Counties saw, on average, a {:.2%} increase in COVID-19 deaths after Biden\'s campaign rallies.'
      .format(df["death_growth_rate_change"].mean()))

Counties saw, on average, a 556.52% increase in the rate of COVID-19 cases from before Biden's campaign rallies to after rallies.
Counties saw, on average, a 13.48% increase in COVID-19 deaths after Biden's campaign rallies.


In [43]:
df = pd.read_sql_query('SELECT * FROM rally WHERE candidate_id = 2', con=engine) # Trump

print('Counties saw, on average, a {:.2%} increase in the rate of COVID-19 cases from before Trump\'s rallies to after rallies.'
      .format(df["case_growth_rate_change"].mean()))
print('Counties saw, on average, a {:.2%} increase in COVID-19 deaths after Trump\'s campaign rallies.'
      .format(df["death_growth_rate_change"].mean()))

Counties saw, on average, a 499.96% increase in the rate of COVID-19 cases from before Trump's rallies to after rallies.
Counties saw, on average, a 15.89% increase in COVID-19 deaths after Trump's campaign rallies.
